**Here i have used XGBRegressor to predict the prices. The RMSE values can be improved upon but this just a starter pack for anywhere who wants to work with XGB. I have one hot encoded categorical variables and have also droped some attributes. Ideally, the longtitude and latitude attribute could have been handled better. Also using stacked models may improve results.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
datatrain = pd.read_csv("/kaggle/input/us-airbnb-open-data/AB_US_2020.csv")

In [ ]:
datatrain.head()

In [ ]:
datatrain.describe()

We set the index as "id"

In [ ]:
datatrain.set_index("id")

In [ ]:
plt.rcParams['figure.figsize']=12,12
g = sns.heatmap(datatrain.corr(),annot=True)

In [ ]:
datatrain["neighbourhood"].nunique()

In [ ]:
datatrain["neighbourhood"].unique()

We drop attributes like "last_review", "host_name" and "name" as they would not correlate to the prices. "Host_name" is dropped as there is already a "host_id" column which is used 

In [ ]:
datatrain.drop("last_review",inplace=True,axis=1)
datatrain.drop("host_name",inplace=True,axis=1)

In [ ]:
datatrain.drop("name",inplace=True,axis=1)

We create a new class for neighbourhood group with "null" and set all of them as 0

In [ ]:
datatrain.neighbourhood_group=datatrain.neighbourhood_group.fillna(0)

In [ ]:
datatrain.neighbourhood_group.value_counts(0)

In [ ]:
datatrain.neighbourhood.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
datatrain['neighbourhood'] = labelencoder.fit_transform(datatrain["neighbourhood"])
datatrain

In [ ]:
datatrain.describe()

In [ ]:
datatrain.room_type.value_counts()

In [ ]:
y = pd.get_dummies(datatrain.room_type, prefix='RT')
datatrain = datatrain.join(y)

In [ ]:
y = pd.get_dummies(datatrain.neighbourhood_group, prefix='NBG')
datatrain = datatrain.join(y)

In [ ]:
datatrain.drop("neighbourhood_group",axis=1,inplace=True)

In [ ]:
datatrain["reviews_per_month"].fillna(datatrain.mean(),inplace=True)

In [ ]:
datatrain["reviews_per_month"].value_counts()

In [ ]:
datatrain.drop("room_type",axis=1,inplace=True)

In [ ]:
datatrain.describe().columns

In [ ]:
datatrain.columns

In [ ]:
y = pd.get_dummies(datatrain.city)
datatrain = datatrain.join(y)
datatrain.drop("city",axis=1,inplace=True)

In [ ]:
datatrain

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X=datatrain.drop("price",axis=1)
y = datatrain.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0,2],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb = XGBRegressor()
rs = RandomizedSearchCV(xgb, param_distributions=params, n_iter=5, n_jobs=4, cv=5, verbose=3, random_state=1001 )
rs.fit(X_train, y_train)

In [ ]:
y_pred0 = rs.best_estimator_.predict(X_train)
y_pred = rs.best_estimator_.predict(X_test)
print(np.sqrt(mean_squared_error(y_train, y_pred0)))
print(np.sqrt(mean_squared_error(y_test, y_pred)))